# Innledning
I denne computational essayen for FYS1120 skal vi utdype oss, prinsippet bak samt med historien rundt Solenoiden. Videre skal vi se på hovedbudskapet om at en solenoide faktisk har null felter, både det magnetiske og det elektriske feltet i systemet, ved å visualisere og tilpasse variablene. Vi skal utdype metoden samt fremgangsmåten for hvordan vi skal finne ut av problemstillingen til prosjektet vårt. Dette arbeidet vil foregå i samarbeid med en annen, men dokumentasjonen og kodingen vil være hensiktsmessig tilpasset min forståelse av oppgaven. Dette gjelder også utviklingen av KI i nåværende samfunn, der det brukes til å dobbeltsjekke faktiske opplysninger. Det ble brukt __grammatiske KI-verktøy__ for å minske feil i grammatikken eller budskapet i essayen.

## Formålet med essayen
Formålet med denne essayen er å besvare spørsmål som dreier seg om hvordan en solenoide fungerer. Blant de spørsmålene man kan komme på med, er det disse spørsmålene som skal utforskes og besvares i denne essayen:

 - Er det mulig å finne ut om det magnetiske feltet fra en solenoide går mot null? I så fall, under hvilke betingelser?
 - Hvilke faktorer påvirker styrken til det magnetiske feltet fra en solenoide?
 - Hvordan kan beregningsmodeller og simuleringer brukes til å undersøke forholdene der det magnetiske feltet går mot null? 

## Solenoide -- kort forklart

Solenoiden er en isolert leder bygget som en tettviklet spole. Når strøm tilføres systemet, vil den danne et elektromagnetisk felt rundt solenoiden. Det dannes altså et magnetisk felt rundt og langs aksen til solenoiden. Dette kan brukes til å danne om elektrisk energi til mekanisk energi ved bruk av magnetisk kraft. En solenoide er i essens en spole og kan dermed brukes til å lagre energi i magnetfeltet den danner.

Det finnes 2 foskjellige soleniode som ekstierer i markende det enten "valve eller electric". Begge i prinsippe har egenskapen ved å dytte eller hente materialen den er polet til Solenoiden. Komponenten kan brukes i en rekke forskjellige systemer, det er mulig å bruke solenoide som endringer mellom forskjellige spenninger ved bygging av transformatorer og generatorer for generering av strøm.

%%latex
\newpage

# Fremgangsmåte
I denne *Computational Essay* skal det kjøres simuleringer i Python av magnetfeltet fra en solenoide sett fra tre forskjellige planer: $(x,y)$-, $(y,z)$- og $(x,z)$-planet. Det magnetiske feltet rundt en solenoide skal også utregnes. Deretter skal det kjøres et seperat program som leter etter mørke soner i alle tre planer. En *rød* sone er altså der den magnetiske feltstyrken $\|\mathbf{B}\|$ er lik null.

Det er hovedsakelig kun én størrelse som blir brukt i dette prosjektet: det magnetiske feltet $\mathbf{B}$.

## Magnetisk felt fra en strømførende leder (Biot-Savarts lov)
For en strømførende leder med kurve $C$, så er det magnetiske feltet
sett fra et vilkårlig punkt $\mathbf{r}$ i rommet gitt ved

\begin{align*}
\mathbf{B}(\mathbf{r}) = \frac{\mu_0}{4\pi} \int_C{\frac{I \, d\mathbf{l} \times \left(\mathbf{r} - \mathbf{l}\right)}{\left\|\mathbf{r} - \mathbf{l}\right\|^3}}
\end{align*}

der $I$ er strømmen, $d\mathbf{l}$ er linjeelementet til
lederen og $\mu_0$ er vakuumpermeabiliteten. Dette uttrykket vil være nyttig
da en spiralkurve best tilnærmer en solenoide.


## Kode for simulasjoner i planer


# Kan endringer i soleniodens størrelse skape nuller i magnetisk feltstyrke?
Det ble sett gjennom hvordan det var i helt standardisert størrelse. Men vil det få en null magnetfelt hvis vi gjør den større eller mindre på solenoiden? Hvis vi utdyper oss den formelen vi har brukt i for mangetiske feltet 
$$
\mathbf{B}(\mathbf{r}) = \frac{\mu_0}{4\pi} \int_C \frac{I \, d\mathbf{\ell} \times \left(\mathbf{r} - \mathbf{\ell}\right)}{\|\mathbf{r} - \mathbf{\ell}\|^3}
$$
Vil det tar hensyn med lengden til over tid. Dette vil skal større B felt som igjen kan føre til en dø sone eten gjennom den indre eller ytre felter. Skal vi bruke den animasjon plotting der det kan ses forksjeller i B feltet i XY og XZ ved endringer mellom XZ og YZ lite for holde seg til. Det måten som trengs gjøre bare separere seg Radiusen og lengden som en liste og plotte som en for loop med de to listene. Der den simulere ut tre forskjellige størrelse eten i lengden eller radiusen.
    (skriv mer her - Hai)


## Kode for en solenoide med varierende størrelse

# Hva skjer dersom man kobler solenoider i serie og parallell?
Ettersom at solenoiden i normal og variert skapte interessant endringer i XY. Er det noe feil vi fått. Det akn gjøre prøve supplere ut som delt opp serie og paraelle kobling i solenoider. Dette  kan indikere at vår B-felt funskjon i den 0 område kan sette utslag der det ikke skal være. Dette gjør av lage to objekter i lik størrelse ( i denne tilfelle er de halvert i to for en sammenlagt solenoide). Der plottes inn ved bruk av plt.ledgend.   ( skriv resultatet her). Er det mulighet lage da serie og paraellekoblinger til null B felt? 

## Kode til serie- og parallellkoblede solenoider

# Diskusjon
Jeg aner ikke hva jeg skal skrive her... - J (Bruh, her skriver man hva som ble funnet i systemene vi har laget , som er viktig se hva som er funnet og videre)

vi ser ut ifra dataene vi har funnet ved den serie se at den tass inn imidten . Som gis riktig ved de to solonoidene i mellom de to .... ( skriv mer i seriekobling)

# Konklusjon
Fra det vi har funnet om det er mulig at det finnes null $\mathbf{B}$- felt i hele systemet så ble det oppdaget at det det kan ikke fullsyedig konkludere ved nåværende PC pga. kapasistet og begrensinger PC kan kakluere seg frem til mer sikkre svar. Fra det som fikk ut av systemet kan det settet ikke kan det ikke noe 0 B felt unntatt i noe XY i solenoide.  

De rekke mulige feilkilder som kan medfører usikkheter eller kritiske påvirkninger til simuleringer vi har funnet. Det settes som sjekkliste under. Men hoved problemer gjenstår gjennom den kalkulasjonen med 0 B-felt, Denne presentere store deler av prosjeketet og ser store usikkerheter i XY planet spesielt. Denne Kan være at vi skjærer fra den XY der Z = 0 på de to første vanlige og variert. Det som medfører avik i indre og ytre. Det kan sies at utenfor det er jo XZ og YZ riktig ved at det er kalt inn som lukket systemet. Som si at hele summen av B-felt vil gå mot 0. Som da ses to forskjellige feltstyrker.  

(skriv mer her)

## Kildeføring
- Fakta om Solenoide - https://snl.no/solenoid
- Utvidelse av Soleniode - https://ccoils.com/blog/what-is-a-solenoid/ 